In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import gmaps.datasets
import requests
# Import the API key.
from config import g_key
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Mortka,99,RU,2020-08-18 00:55:40,99,59.33,66.02,50.72,5.23
1,1,Plettenberg Bay,100,ZA,2020-08-18 00:55:41,80,-34.05,23.37,47.03,22.93
2,2,Caravelas,0,BR,2020-08-18 00:55:43,86,-17.71,-39.25,72.12,6.17
3,3,Iranshahr,87,IR,2020-08-18 00:55:46,26,27.20,60.68,95.00,4.70
4,4,Athabasca,73,CA,2020-08-18 00:52:11,65,54.72,-113.29,80.04,4.09


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)

# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Caravelas,BR,72.12,-17.71,-39.25,
5,Samarai,PG,76.60,-10.62,150.67,
10,Kapit,MY,79.43,2.02,112.93,
15,Rikitea,PF,72.14,-23.12,-134.97,
22,Albany,US,71.01,42.60,-73.97,
...,...,...,...,...,...,...
519,Angoche,MZ,70.97,-16.23,39.91,
520,Cabrobo,BR,75.15,-8.51,-39.31,
525,Neiafu,TO,75.20,-18.65,-173.98,
530,Pottstown,US,71.60,40.25,-75.65,


In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &  (city_data_df["Max Temp"] >= min_temp)]
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [38]:
import time
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    }

#Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
   

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   
    try:
        # Grab the first hotel from the results and store the name.
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    #print(hotels["results"][0]["name"])
    except(IndexError):
        hotel_df.loc[index, "Hotel Name"]=np.nan  
        print("Hotel not found... skipping.")  


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [39]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Caravelas,BR,72.12,-17.71,-39.25,Pousada dos Navegantes
5,Samarai,PG,76.60,-10.62,150.67,Nuli Sapi
10,Kapit,MY,79.43,2.02,112.93,Hotel Meligai Kapit
15,Rikitea,PF,72.14,-23.12,-134.97,Pension Maro'i
22,Albany,US,71.01,42.60,-73.97,NaN
...,...,...,...,...,...,...
519,Angoche,MZ,70.97,-16.23,39.91,Guest House HESADA
520,Cabrobo,BR,75.15,-8.51,-39.31,Hotel Limarques
525,Neiafu,TO,75.20,-18.65,-173.98,Mystic Sands
530,Pottstown,US,71.60,40.25,-75.65,Comfort Inn & Suites Pottstown - Limerick


In [40]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
#Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))